# Test Distance Based Localization on real cases

In [1]:
import numpy as np
import polars as pl
import matplotlib.pyplot as plt

from iterative_ensemble_smoother.experimental import DistanceESMDA
from iterative_ensemble_smoother.esmda import ESMDA

from ert.storage import open_storage

SEED = 42

## List all available experiments in storage

In [2]:
storage_path = "01_drogon_ahm/storage/"
storage_path = "/Users/FCUR/git/ert/test-data/ert/heat_equation/storage"
with open_storage(storage_path) as storage:
    [print(f"Experiment names: {x.name}") for x in storage.experiments]

Experiment names: ensemble_smoother


## Pick which experiment to analyse

In [3]:
experiment_name = "ensemble_smoother"

## Load observations and responses from storage. Remove responses with zero standard deviation

In [4]:
with open_storage(storage_path, "r") as storage:
    ensemble = storage.get_experiment_by_name(experiment_name).get_ensemble_by_name("default_0")
    ensemble_size = ensemble.ensemble_size
    selected_obs = ensemble.experiment.observation_keys
    iens_active_index = np.array(ensemble.get_realization_list_with_responses())
    observations_and_responses = ensemble.get_observations_and_responses(
        selected_obs,
        iens_active_index
    )

response_cols = [str(i) for i in range(1, ensemble.ensemble_size)]
df_filtered = observations_and_responses.filter(
    pl.concat_list([pl.col(col) for col in response_cols])
    .list.eval(pl.element().std())
    .list.first() > 0
)

In [5]:
df_filtered

response_key,index,observation_key,observations,std,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,…,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
str,str,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
"""MY_RESPONSE""","""10, 0""","""MY_OBS_10""",0.000006,3.0797e-7,0.001738,0.014287,0.010067,0.000116,0.010186,0.000495,0.000006,0.00013,0.00004,0.000075,0.000006,0.000238,0.007534,0.414858,0.001904,0.032358,0.012611,0.002779,0.000424,3.1249e-7,3.5690e-7,0.019226,0.087599,0.003879,0.001009,0.104038,0.066789,0.058016,0.000435,0.000002,0.000482,0.000004,…,0.000055,0.000011,0.067731,0.014178,1.0160e-7,0.001602,0.024197,0.294349,0.107269,0.000193,0.125961,0.002986,0.003577,0.120661,0.001675,0.014344,0.033997,0.034318,0.000002,0.205756,0.086221,0.001245,0.014565,0.005082,0.001403,0.0041,0.000354,0.00004,0.004931,0.110773,0.011225,0.004557,0.000046,0.000043,0.003056,0.200218,0.320688
"""MY_RESPONSE""","""10, 1""","""MY_OBS_10""",0.414103,0.020195,3.791496,2.388445,0.871504,0.421969,1.882233,0.688758,0.596691,0.484872,1.336216,0.467743,1.080632,0.095577,2.051911,6.71,2.545115,3.15389,5.585956,1.893609,1.815404,0.012714,0.021675,0.709378,1.35977,5.175879,0.934265,4.687365,3.836681,3.277311,2.970897,0.545605,3.226886,0.026443,…,1.800958,3.077124,3.419479,3.460029,0.000833,0.684022,4.070356,7.657216,4.840562,0.622302,5.500516,0.131315,2.700433,3.29128,0.908311,1.06721,2.129442,1.192377,0.055188,2.285549,6.764185,2.664846,1.17452,0.525426,0.82779,0.413551,0.460627,0.563554,2.323675,2.938879,1.765402,1.117418,0.090555,0.300184,1.939161,7.560426,4.939108
"""MY_RESPONSE""","""10, 2""","""MY_OBS_10""",3.667965,0.164823,3.459801,3.073539,0.63332,0.351097,3.379199,1.405518,0.365358,0.505973,1.165848,0.221864,0.482993,0.72843,2.620817,5.616586,1.318776,2.70711,4.190064,4.777524,0.964936,0.076197,0.18081,0.724264,1.39071,1.719255,0.274612,4.245439,4.200376,3.433375,2.375463,0.373046,1.654375,0.025409,…,0.272113,0.697785,2.857745,5.870055,0.00109,0.390619,1.522539,5.38056,2.262373,0.511656,6.197315,0.170156,1.007394,7.109563,3.51579,1.102713,3.551681,1.829301,0.048453,2.283837,2.484502,2.41698,1.283918,0.999563,1.358481,0.400599,0.681584,0.461609,0.935628,3.731048,3.367921,1.654116,2.325458,0.021296,2.648818,4.782776,9.928008
"""MY_RESPONSE""","""10, 3""","""MY_OBS_10""",16.24826,0.771199,9.105598,10.113826,1.697024,2.234903,6.417114,9.410075,5.60522,1.194318,2.799078,2.819279,8.68576,1.591312,4.975577,9.250033,9.135738,4.847055,17.797073,11.836971,3.636055,0.428058,0.789039,1.384356,2.93277,11.879325,5.413805,8.007386,6.607326,5.913062,7.022437,2.710939,8.193822,0.190387,…,7.8991,7.602262,7.835516,16.834829,0.009983,2.993222,12.854205,13.987189,9.731277,4.420914,10.47187,0.330226,10.618836,13.554911,6.804796,3.099226,7.294907,2.113304,0.568262,3.683238,13.128551,9.726973,2.339626,2.438686,4.169963,0.826034,2.224852,2.914973,9.462966,6.189559,9.785872,4.293647,5.162063,1.773264,10.759433,12.389246,6.986765
"""MY_RESPONSE""","""10, 4""","""MY_OBS_10""",1.763592,0.087033,0.098292,0.226188,0.012185,0.012469,1.291371,0.057598,0.000581,0.079196,0.368591,0.021552,0.007176,0.215759,0.948216,0.276918,0.001738,0.498807,0.987291,0.710998,0.001275,0.00513,0.002119,0.215035,0.126447,0.001894,0.000455,0.499554,1.789316,0.660236,0.78219,0.004964,0.162437,0.003221,…,0.000206,0.004396,0.381662,1.142425,0.00008,0.001898,0.042659,0.684418,0.133594,0.00943,2.012809,0.009139,0.040463,0.705422,1.194282,0.022661,0.907232,0.225637,0.018058,0.276068,0.02506,0.460875,0.192472,0.040766,0.157398,0.031884,0.1874,0.132994,0.019102,1.336195,0.304648,0.601241,0.503272,0.000016,0.248767,0.544208,1.628358
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,

# Load parameters from storage

In [16]:
with open_storage(storage_path, "r") as storage:
    experiment = storage.get_experiment_by_name(experiment_name)
    ensemble = experiment.get_ensemble_by_name("default_0")
    groups = list(experiment.parameter_configuration.keys())

    #for group in groups:
    #    print(ensemble.load_parameters_numpy(group, [0, 1]))

    realizations = ensemble.get_realization_list_with_responses()
    cond = ensemble.load_parameters_numpy("COND", realizations)

## Prepare response matrix

In [17]:
Y = df_filtered.select(pl.all().exclude(["response_key", "index", "observation_key", "observations", "std"])).to_numpy()

## Ensemble Smoother without Localization

In [10]:
X = cond

assert Y.shape[1] == X.shape[1]

observations = df_filtered["observations"].to_numpy()
C_D = np.diag(df_filtered["std"])

smoother_ESMDA = ESMDA(
    covariance=C_D,
    observations=observations,
    alpha=1,
    seed=SEED)

D = smoother_ESMDA.perturb_observations(
    ensemble_size=ensemble_size, alpha=1
)

X_posterior_ESMDA = smoother_ESMDA.assimilate(X=X, Y=Y)

## Ensemble Smoother with Distance Based Localization

In [14]:
smoother = DistanceESMDA(
    covariance=C_D,
    observations=observations,
    alpha=1,
    seed=SEED
)

rho = np.ones(shape=(X.shape[0], Y.shape[0]))

X_posterior = smoother.assimilate(X=X, Y=Y, rho=rho)